# Feature engineering

Es el proceso de utilizar el conocimiento del dominio para transformar algunas variables y enriquecer el dataset

Estas nuevas variables suelen mejorar el rendimiento de los algoritmos de machine learning

Es aquí donde la creatividad entra en juego

Aquí se presentan algunos métodos, pero

**no es una lista exhaustiva**

Las posibilidades son infinitas

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("./datasets/titanic.csv")
df.head()

,Name,Sex,Age,Pclass,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,"Karaic, Mr. Milan",male,30.0,3,0,0,349246,7.8958,NaN,S,0
1,"Dean, Mrs. Bertram",female,33.0,3,1,2,C.A. 2315,20.5750,NaN,S,1
2,"Rice, Mrs. William",female,39.0,3,0,5,382652,29.1250,NaN,Q,0
3,"Davidson, Mrs. Thornton",female,27.0,1,1,2,F.C. 12750,52.0000,B71,S,1
4,"Ridsdale, Miss. Lucy",female,50.0,2,0,0,W./C. 14258,10.5000,NaN,S,1


In [5]:
df.shape

(1309, 11)

## Combinación/transformación de columnas

Sumas, divisiones, productos... de variables

In [7]:
df["n_familiares"] = df.SibSp + df.Parch

In [8]:
df["is_alone"] = df.n_familiares == 0

In [9]:
df["is_child"] = df.Age < 12

In [10]:
df["is_young"] = df.Age < 21

No hay ideas absurdas. Si algo no funciona, los modelos tipo árbol las desechan

In [11]:
df["years_per_class"] = df.Age / df.Pclass

In [12]:
df.head()

,Name,Sex,Age,Pclass,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,n_familiares,is_alone,is_child,is_young,years_per_class
0,"Karaic, Mr. Milan",male,30.0,3,0,0,349246,7.8958,NaN,S,0,0,True,False,False,10.0
1,"Dean, Mrs. Bertram",female,33.0,3,1,2,C.A. 2315,20.5750,NaN,S,1,3,False,False,False,11.0
2,"Rice, Mrs. William",female,39.0,3,0,5,382652,29.1250,NaN,Q,0,5,False,False,False,13.0
3,"Davidson, Mrs. Thornton",female,27.0,1,1,2,F.C. 12750,52.0000,B71,S,1,3,False,False,False,27.0
4,"Ridsdale, Miss. Lucy",female,50.0,2,0,0,W./C. 14258,10.5000,NaN,S,1,0,True,False,False,25.0


## Tratamiento de strings

Extraer la primera letra de un string

In [16]:
df.Cabin.nunique()

186

In [15]:
df.Cabin.unique()[:20]

array([nan, 'B71', 'B58 B60', 'E67', 'C128', 'D36', 'A20', 'B26', 'B22',
       'D26', 'D33', 'B57 B59 B63 B66', 'C2', 'E50', 'E33', 'B3', 'D15',
       'G6', 'B51 B53 B55', 'C32'], dtype=object)

In [20]:
df["cabin_letter"] = df.Cabin.str[0]

In [23]:
df.cabin_letter.nunique()

8

In [21]:
df.cabin_letter.unique()

array([nan, 'B', 'E', 'C', 'D', 'A', 'G', 'F', 'T'], dtype=object)

In [22]:
df.cabin_letter.value_counts()

cabin_letter
C    94
B    65
D    46
E    41
A    22
F    21
G     5
T     1
Name: count, dtype: int64

Extraer el título del nombre

In [27]:
df["title"] = df.Name.str.split(", ").str[1].str.split(".").str[0]

In [28]:
df.title.value_counts()

title
Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Ms                2
Major             2
Mlle              2
Lady              1
Sir               1
Don               1
Mme               1
Capt              1
Jonkheer          1
Dona              1
the Countess      1
Name: count, dtype: int64

Repito: no hay ideas absurdas. Si algo no funciona, los modelos tipo árbol las desechan

Longitud del nombre: puede indicar status socioeconómico

In [29]:
df["name_length"] = df.Name.str.len()

In [46]:
df["ticket_start_letter"] = df.Ticket.str[0].str.isalpha()

## Relleno de valores nulos

Los modelos de ML no se _comen_ valores nulos

Hay muchas opciones para tratarlos, a priori no sabemos cuál es mejor

Hemos de conocer diferentes estrategias de imputación

El tratamiento de cada columna suele ser diferente

Para montar un modelo sencillo, podemos simplemente dropear las columnas con nulos

Si bien al entrenar un modelo, podríamos dropear las filas con nulos,  
**a la hora de predecir habrá valores nulos que deberemos sí o sí tratar**

In [57]:
df.isna().sum()

Name                      0
Sex                       0
Age                     263
Pclass                    0
SibSp                     0
Parch                     0
Ticket                    0
Fare                      1
Cabin                  1014
Embarked                  2
Survived                  0
n_familiares              0
is_alone                  0
is_child                  0
is_young                  0
years_per_class         263
cabin_letter           1014
title                     0
name_length               0
ticket_start_letter       0
dtype: int64

### variables numéricas

#### con la media

In [58]:
mean_age = df.Age.mean()
mean_age

29.881137667304014

In [59]:
df.Age.fillna(mean_age)
# realmente haríamos esto de abajo, para rellenar la columna Age
# df.Age = df.Age.fillna(mean_age)

0       30.000000
1       33.000000
2       39.000000
3       27.000000
4       50.000000
          ...    
1304    27.000000
1305    26.000000
1306    19.000000
1307    37.000000
1308    29.881138
Name: Age, Length: 1309, dtype: float64

#### con la media por categoría

Por ejemplo, cuál es la media de edad por clase? A cada edad nula, le asignaré la edad media de su clase

In [63]:
pclass_to_mean_age = df.groupby("Pclass").Age.mean().round(1).to_dict()
pclass_to_mean_age

{1: 39.2, 2: 29.5, 3: 24.8}

In [65]:
df.loc[df.Age.isna(), "Pclass"].map(pclass_to_mean_age)
# df.loc[df.Age.isna(), "Age"] = df.loc[df.Age.isna(), "Pclass"].map(pclass_to_mean_age)

6       24.8
7       29.5
13      29.5
24      24.8
29      39.2
        ... 
1278    24.8
1285    24.8
1288    29.5
1302    39.2
1308    24.8
Name: Pclass, Length: 263, dtype: float64

#### con 0 u otro valor fijo

Por ejemplo, en un dataset con una columna `number_children` pueden cambiarse nulos por 0s

In [67]:
df.head()

,Name,Sex,Age,Pclass,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,n_familiares,is_alone,is_child,is_young,years_per_class,cabin_letter,title,name_length,ticket_start_letter
0,"Karaic, Mr. Milan",male,30.0,3,0,0,349246,7.8958,NaN,S,0,0,True,False,False,10.0,NaN,Mr,17,False
1,"Dean, Mrs. Bertram",female,33.0,3,1,2,C.A. 2315,20.5750,NaN,S,1,3,False,False,False,11.0,NaN,Mrs,18,True
2,"Rice, Mrs. William",female,39.0,3,0,5,382652,29.1250,NaN,Q,0,5,False,False,False,13.0,NaN,Mrs,18,False
3,"Davidson, Mrs. Thornton",female,27.0,1,1,2,F.C. 12750,52.0000,B71,S,1,3,False,False,False,27.0,B,Mrs,23,True
4,"Ridsdale, Miss. Lucy",female,50.0,2,0,0,W./C. 14258,10.5000,NaN,S,1,0,True,False,False,25.0,NaN,Miss,20,True


#### vecinos cercanos

Puedo construir una métrica de similaridad entre filas, para luego

asignar a cada edad nula la edad media de los N vecinos más cercanos

### variables categóricas

In [68]:
df.isna().sum()

Name                      0
Sex                       0
Age                     263
Pclass                    0
SibSp                     0
Parch                     0
Ticket                    0
Fare                      1
Cabin                  1014
Embarked                  2
Survived                  0
n_familiares              0
is_alone                  0
is_child                  0
is_young                  0
years_per_class         263
cabin_letter           1014
title                     0
name_length               0
ticket_start_letter       0
dtype: int64

#### con la moda

In [78]:
df.cabin_letter.isna().sum()

1014

In [72]:
df.cabin_letter.value_counts()

cabin_letter
C    94
B    65
D    46
E    41
A    22
F    21
G     5
T     1
Name: count, dtype: int64

In [77]:
top_cabin = df.cabin_letter.mode()[0]
top_cabin

'C'

In [ ]:
df.cabin_letter.fillna(top_cabin)
# df.cabin_letter = df.cabin_letter.fillna(top_cabin)

#### con la moda por categoría

In [81]:
def get_mode(series):
    return series.cabin_letter.mode()[0]

In [82]:
pclass_to_top_cabin = df.groupby("Pclass").apply(get_mode)

In [83]:
pclass_to_top_cabin

Pclass
1    C
2    F
3    F
dtype: object

In [ ]:
df.loc[df.cabin_letter.isna(), "Pclass"].map(pclass_to_top_cabin)
# df.loc[df.cabin_letter.isna(), "cabin_letter"] = df.loc[df.cabin_letter.isna(), "Pclass"].map(pclass_to_top_cabin)

#### category "Unknown"

In [85]:
df.cabin_letter.value_counts()

cabin_letter
C    94
B    65
D    46
E    41
A    22
F    21
G     5
T     1
Name: count, dtype: int64

In [ ]:
df.cabin_letter.fillna("U")
# df.cabin_letter = df.cabin_letter.fillna("U")

### Más estrategias

#### modelo predictivo

Utilizar un modelo de regresión/clasificación utilizando:
 - otras features como predictoras
 - la columna numérica/categórica como columna objetivo

[Null Values Imputations Strategies](https://www.kaggle.com/discussions/general/248836)

## Categorical encoding

Los modelos de ML no se _comen_ variables categóricas

Hay muchas opciones para tratarlas, y a priori no sabemos cuál es mejor

Hemos de conocer estrategias de transformación **categórico --> numérico**

El tratamiento de cada columna suele ser diferente

Podemos tratar una columna de varias maneras: hacer get_dummies y además mean encoding

Para montar un modelo sencillo, podemos simplemente dropear las columnas categóricas

### Texto binario a booleano

In [88]:
df.Sex.map({"male": 0, "female": 1})

0       0
1       1
2       1
3       1
4       1
       ..
1304    0
1305    0
1306    0
1307    0
1308    1
Name: Sex, Length: 1309, dtype: int64

### One Hot Encoder (get_dummies)

Se utiliza cuando (orientativamente):
 * no hay un orden natural entre las categorías y
 * no hay un número enorme (~>20) de categorías

In [89]:
df.Embarked.value_counts()

Embarked
S    914
C    270
Q    123
Name: count, dtype: int64

In [103]:
dummies = pd.get_dummies(df.Embarked, prefix="Embarked")

In [104]:
df.Embarked.head()

0    S
1    S
2    Q
3    S
4    S
Name: Embarked, dtype: object

In [105]:
dummies.head()

,Embarked_C,Embarked_Q,Embarked_S
0,False,False,True
1,False,False,True
2,False,True,False
3,False,False,True
4,False,False,True


In [106]:
df = pd.concat([df.drop('Embarked', axis=1), dummies], axis=1)

In [107]:
df.columns

Index(['Name', 'Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Survived', 'n_familiares', 'is_alone', 'is_child', 'is_young',
       'years_per_class', 'cabin_letter', 'title', 'name_length',
       'ticket_start_letter', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

### Label Encoder

Se utiliza cuando:
 * hay un número no pequeño de categorías
 * se les da _erroneamente_ un orden

In [108]:
df.cabin_letter.unique()

array([nan, 'B', 'E', 'C', 'D', 'A', 'G', 'F', 'T'], dtype=object)

In [109]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [110]:
df.cabin_letter = le.fit_transform(df.cabin_letter)

In [111]:
df.cabin_letter.unique()

array([8, 1, 4, 2, 3, 0, 6, 5, 7])

### Frequency Encoder

Asigna a cada categoría su popularidad

In [113]:
df["cabin_letter2"] = df.Cabin.str[0]

In [135]:
cabin_to_frequency = {name: i for i, name in enumerate(df.cabin_letter2.value_counts().index)}

In [136]:
cabin_to_frequency

{'C': 0, 'B': 1, 'D': 2, 'E': 3, 'A': 4, 'F': 5, 'G': 6, 'T': 7}

In [ ]:
df.cabin_letter2.map(cabin_to_frequency)

### Ordinal Encoder

Se utiliza cuando:
 * sí hay un orden natural entre las categorías

In [120]:
dff = pd.DataFrame({"name": ["Juan", "Pepe", "Marta", "Luna", "Ricardo", "Rosa"], "velocity": ["Fast", "Slow", "Very fast", "Slow", "Average", "Very slow"]})

In [121]:
dff

,name,velocity
0,Juan,Fast
1,Pepe,Slow
2,Marta,Very fast
3,Luna,Slow
4,Ricardo,Average
5,Rosa,Very slow


In [122]:
from sklearn import preprocessing
oe = preprocessing.OrdinalEncoder(categories=[['Very slow', 'Slow', 'Average', 'Fast', 'Very fast']])

In [123]:
dff.velocity = oe.fit_transform(dff[["velocity"]])

In [124]:
dff

,name,velocity
0,Juan,3.0
1,Pepe,1.0
2,Marta,4.0
3,Luna,1.0
4,Ricardo,2.0
5,Rosa,0.0


### Target Encoding

Bastante potente

Asigna a cada categoría la media de la variable objetivo de los de esa categoría

Se utiliza cuando:
 * hay muchas categorías

In [125]:
df.title.value_counts()

title
Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Ms                2
Major             2
Mlle              2
Lady              1
Sir               1
Don               1
Mme               1
Capt              1
Jonkheer          1
Dona              1
the Countess      1
Name: count, dtype: int64

In [129]:
title_to_prob_survived = df.groupby("title").Survived.mean().round(3).sort_values().to_dict()
title_to_prob_survived

{'Capt': 0.0,
 'Don': 0.0,
 'Jonkheer': 0.0,
 'Rev': 0.0,
 'Mr': 0.162,
 'Dr': 0.5,
 'Major': 0.5,
 'Col': 0.5,
 'Ms': 0.5,
 'Master': 0.508,
 'Miss': 0.677,
 'Mrs': 0.787,
 'Lady': 1.0,
 'Dona': 1.0,
 'Mme': 1.0,
 'Mlle': 1.0,
 'Sir': 1.0,
 'the Countess': 1.0}

In [131]:
df.title = df.title.map(title_to_prob_survived)

## Notas

A los árboles les da igual la magnitud, solo el orden

Puede haber categorías en el test que no estén en el train

Antes de entrenar, asegúrate de que:
 - no hay nulos
 - solo hay columnas numéricas

In [132]:
df.isna().sum()

Name                      0
Sex                       0
Age                     263
Pclass                    0
SibSp                     0
Parch                     0
Ticket                    0
Fare                      1
Cabin                  1014
Survived                  0
n_familiares              0
is_alone                  0
is_child                  0
is_young                  0
years_per_class         263
cabin_letter              0
title                     0
name_length               0
ticket_start_letter       0
Embarked_C                0
Embarked_Q                0
Embarked_S                0
cabin_letter2          1014
dtype: int64

In [133]:
df.dtypes

Name                    object
Sex                     object
Age                    float64
Pclass                   int64
SibSp                    int64
Parch                    int64
Ticket                  object
Fare                   float64
Cabin                   object
Survived                 int64
n_familiares             int64
is_alone                  bool
is_child                  bool
is_young                  bool
years_per_class        float64
cabin_letter             int64
title                  float64
name_length              int64
ticket_start_letter       bool
Embarked_C                bool
Embarked_Q                bool
Embarked_S                bool
cabin_letter2           object
dtype: object

Antes de predecir, asegúrate de que:
 - no hay nulos
 - solo hay columnas numéricas

In [ ]:
# df_test.isna().sum()

In [ ]:
# df_test.dtypes

Antes de predecir, asegúrate de que:  
**el orden de las columnas del test es como el train** (exceptuando la variable objetivo)

Puedes usar algo similar a:

In [134]:
cols_train = df.drop(columns="Survived").columns.tolist()
cols_train

['Name',
 'Sex',
 'Age',
 'Pclass',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'n_familiares',
 'is_alone',
 'is_child',
 'is_young',
 'years_per_class',
 'cabin_letter',
 'title',
 'name_length',
 'ticket_start_letter',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S',
 'cabin_letter2']

In [ ]:
# df_test = df_test[cols_train]

Algunos ejemplos y la respuesta de ChatGPT:

Encoding de géneros de películas:
 - One-Hot Encoding: Ideal si no hay demasiados géneros y cada película pertenece a un solo género.

Encoding de días de la semana:
 - Ordinal Encoding: Dado que los días de la semana tienen un orden natural (lunes a domingo).
 - One-Hot Encoding: Si no se considera el orden de los días y se trata cada día como una categoría independiente.

Encoding de estaciones del año:
 - Ordinal Encoding: Las estaciones siguen un orden cíclico y pueden codificarse secuencialmente.
 - One-Hot Encoding: Si se prefiere tratar cada estación como una categoría independiente.

Encoding de códigos postales:
 - Target Encoding: Útil si hay muchos códigos postales y están correlacionados con la variable objetivo.
 - Frequency Encoding: Codifica los códigos postales según la frecuencia de su aparición en el dataset.

Encoding de niveles educativos:
 - Ordinal Encoding: Ya que los niveles educativos tienen un orden inherente (por ejemplo, primaria, secundaria, universitaria).
 - One-Hot Encoding: Si se considera cada nivel educativo como una categoría separada sin orden.

Encoding de países del mundo:
 - One-Hot Encoding: Adecuado si la lista de países no es excesivamente larga y se trata cada país como una categoría única.
 - Less categories encoding: pasarlo a continente
 - Target encoding: asignar la media de la variable objetivo a cada país

# Hiperparámetros: encontrando los mejores

Los modelos aceptan diferentes hiperparámetros

In [148]:
from sklearn.tree import DecisionTreeClassifier

In [149]:
tree = DecisionTreeClassifier(max_depth=5)

In [150]:
tree.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

Cómo encuentro los mejores?

## GridSearch

GridSearchCV:
 - realiza una **búsqueda exhaustiva** sobre un subconjunto especificado del espacio de hiperparámetros
 - puede encontrar la mejor combinación de hiperparámetros
 - puede ser muy lento con un gran número de combinaciones de hiperparámetros

In [151]:
from sklearn.model_selection import GridSearchCV

In [154]:
df.columns

Index(['Name', 'Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Survived', 'n_familiares', 'is_alone', 'is_child', 'is_young',
       'years_per_class', 'cabin_letter', 'title', 'name_length',
       'ticket_start_letter', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'cabin_letter2'],
      dtype='object')

In [166]:
dff = df[['Sex', 'Age', 'Pclass', 'Fare',
        'n_familiares', 'is_alone', 'is_child',
       'years_per_class', 'cabin_letter', 'title', 'name_length',
       'ticket_start_letter', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Survived']].copy()

In [167]:
dff.Age = dff.Age.fillna(dff.Age.mean())

In [168]:
dff.years_per_class = dff.years_per_class.fillna(0)

In [170]:
dff.Fare = dff.Fare.fillna(0)

In [173]:
dff.head()

,Sex,Age,Pclass,Fare,n_familiares,is_alone,is_child,years_per_class,cabin_letter,title,name_length,ticket_start_letter,Embarked_C,Embarked_Q,Embarked_S,Survived
0,0,30.0,3,7.8958,0,True,False,10.0,8,0.162,17,False,False,False,True,0
1,1,33.0,3,20.5750,3,False,False,11.0,8,0.787,18,True,False,False,True,1
2,1,39.0,3,29.1250,5,False,False,13.0,8,0.787,18,False,False,True,False,0
3,1,27.0,1,52.0000,3,False,False,27.0,1,0.787,23,True,False,False,True,1
4,1,50.0,2,10.5000,0,True,False,25.0,8,0.677,20,True,False,False,True,1


In [183]:
params = {
    'criterion': ['gini', 'log_loss'],
    'max_depth': [4, 5, 6, 7, 8, 9],
    'min_samples_split': [2, 5, 10],
    'max_features': [0.8, 1]
}

In [198]:
gs = GridSearchCV(
    estimator=DecisionTreeClassifier(), 
    param_grid=params, 
    cv=5,
    # n_jobs=-2, 
    verbose=1,
    return_train_score=True
)

In [199]:
X = dff.drop(columns="Survived")
y = dff.Survived

In [200]:
%%time
gs.fit(X, y)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
CPU times: user 2.57 s, sys: 0 ns, total: 2.57 s
Wall time: 2.59 s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'log_loss'],
                         'max_depth': [4, 5, 6, 7, 8, 9],
                         'max_features': [0.8, 1],
                         'min_samples_split': [2, 5, 10]},
             return_train_score=True, verbose=1)

In [201]:
gs.best_params_

{'criterion': 'log_loss',
 'max_depth': 5,
 'max_features': 0.8,
 'min_samples_split': 10}

In [202]:
gs.best_estimator_

DecisionTreeClassifier(criterion='log_loss', max_depth=5, max_features=0.8,
                       min_samples_split=10)

In [203]:
cv_results = pd.DataFrame(gs.cv_results_)[['params', 'mean_test_score', 'mean_train_score']]

In [205]:
cv_results.head()

,params,mean_test_score,mean_train_score
0,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",0.798321,0.829832
1,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",0.799087,0.831550
2,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",0.799090,0.832123
3,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",0.724234,0.747138
4,"{'criterion': 'gini', 'max_depth': 4, 'max_fea...",0.714305,0.742360


In [ ]:
gs.predict(X_test)

## RandomSearch

RandomSearchCV:
 - selecciona **al azar** combinaciones de hiperparámetros para probar
 - más rápido que GridSearchCV, ya que no prueba todas las combinaciones posibles
 - es eficaz si la dimensión del espacio de hiperparámetros es alta
 - no garantiza encontrar la mejor combinación de hiperparámetros, ya que la búsqueda es aleatoria.

In [206]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

[CV 3/5] END criterion=log_loss, max_depth=6, max_features=1, min_samples_split=10;, score=0.760 total time=   0.0s
[CV 4/5] END criterion=log_loss, max_depth=6, max_features=1, min_samples_split=10;, score=0.710 total time=   0.0s
[CV 5/5] END criterion=log_loss, max_depth=6, max_features=1, min_samples_split=10;, score=0.747 total time=   0.0s
[CV 1/5] END criterion=log_loss, max_depth=7, max_features=0.8, min_samples_split=2;, score=0.790 total time=   0.0s
[CV 2/5] END criterion=log_loss, max_depth=7, max_features=0.8, min_samples_split=2;, score=0.809 total time=   0.0s
[CV 3/5] END criterion=log_loss, max_depth=7, max_features=0.8, min_samples_split=2;, score=0.805 total time=   0.0s
[CV 4/5] END criterion=log_loss, max_depth=7, max_features=0.8, min_samples_split=2;, score=0.748 total time=   0.0s
[CV 5/5] END criterion=log_loss, max_depth=7, max_features=0.8, min_samples_split=2;, score=0.808 total time=   0.0s
[CV 1/5] END criterion=log_loss, max_depth=7, max_features=0.8, min

In [207]:
param_dist = {
    "max_depth": list(range(3, 20)),
    "max_features": list(range(1, 9)),
    "min_samples_leaf": list(range(1, 30, 2)),
    "criterion": ["gini", "log_loss", "entropy"]
}

In [208]:
param_dist

{'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 'max_features': [1, 2, 3, 4, 5, 6, 7, 8],
 'min_samples_leaf': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29],
 'criterion': ['gini', 'log_loss', 'entropy']}

In [223]:
random_search = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions=param_dist,
                                   n_iter=200, cv=5, random_state=42, return_train_score=True)

In [224]:
random_search.fit(X, y)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=200,
                   param_distributions={'criterion': ['gini', 'log_loss',
                                                      'entropy'],
                                        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10,
                                                      11, 12, 13, 14, 15, 16,
                                                      17, 18, 19],
                                        'max_features': [1, 2, 3, 4, 5, 6, 7,
                                                         8],
                                        'min_samples_leaf': [1, 3, 5, 7, 9, 11,
                                                             13, 15, 17, 19, 21,
                                                             23, 25, 27, 29]},
                   random_state=42, return_train_score=True)

In [225]:
random_search.best_params_

{'min_samples_leaf': 9,
 'max_features': 5,
 'max_depth': 10,
 'criterion': 'log_loss'}

In [226]:
random_search.best_estimator_

DecisionTreeClassifier(criterion='log_loss', max_depth=10, max_features=5,
                       min_samples_leaf=9)

In [227]:
cv_results_randomsearch = pd.DataFrame(random_search.cv_results_)[['params', 'mean_test_score', 'mean_train_score']]

In [228]:
cv_results_randomsearch.sort_values("mean_test_score").head()

,params,mean_test_score,mean_train_score
80,"{'min_samples_leaf': 7, 'max_features': 1, 'ma...",0.691410,0.682385
104,"{'min_samples_leaf': 27, 'max_features': 1, 'm...",0.701263,0.715624
178,"{'min_samples_leaf': 25, 'max_features': 1, 'm...",0.708090,0.744477
172,"{'min_samples_leaf': 5, 'max_features': 1, 'ma...",0.711237,0.721164
45,"{'min_samples_leaf': 29, 'max_features': 1, 'm...",0.715007,0.738932


<img width=600 src="https://miro.medium.com/v2/resize:fit:1004/0*yDmmJmvRowl0cSN8.png">